In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_validate
import pandas as pd
from sklearn.model_selection import GridSearchCV
from loguru import logger as lg
import multiprocessing
import nltk
from nltk.corpus import stopwords
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
import warnings
warnings.filterwarnings("ignore")
# Download necessary NLTK corpora
# nltk.download('stopwords')

In [2]:
NUM_PROCESSORS = (multiprocessing.cpu_count() - 2)
VERBOSE = 0
DEPARTMENTS_QUANTITY_TO_TRAIN = 6
lg.add("finnlp_binary_intra_class_experiment_2.log", rotation="1024 MB")
lg.info("\n\n####################################### NEW EXECUTION #######################################\n\n")
lg.info(f"Processors (cores) available: {NUM_PROCESSORS}")

2024-03-25 22:04:21.531 | INFO     | __main__:<module>:5 - 

####################################### NEW EXECUTION #######################################


2024-03-25 22:04:21.532 | INFO     | __main__:<module>:6 - Processors (cores) available: 10


In [3]:
bbrc = pd.read_csv("BBRC_Brazilian_Banking_Regulation_Corpora.csv" ,encoding="utf-8", low_memory=False, sep=";")

In [4]:
bbrc.head(20)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id
0,1,usi,2020-12-23,2128,36560,2020-12-23,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,656,67.0,127.0,COMUNICADO,771984
1,1,usi,2020-12-28,2129,36569,2020-12-28,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,2110,156.0,322.0,COMUNICADO,772973
2,1,usi,2020-12-22,2130,36554,2020-12-22,BACEN,Divulga comunicado do Grupo de Ação Financeira...,120,16.0,18.0,Comunicamos que o Grupo de Ação Financeira con...,792,76.0,127.0,COMUNICADO,771452
3,1,usi,2020-12-23,2131,4880,2020-12-23,BACEN,Dispõe sobre o horário de atendimento ao públi...,222,32.0,34.0,"O Banco Central do Brasil, na forma do art. 9º...",4964,365.0,875.0,RESOLUÇÃO CMN,772051
4,1,usi,2020-12-23,2132,1125/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,165,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,3382,304.0,492.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771966
5,1,usi,2020-12-23,2133,1120/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,169,19.0,24.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,3201,287.0,480.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771967
6,1,usi,2020-12-23,2134,1122/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,169,19.0,24.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,3215,289.0,482.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771968
7,1,usi,2020-12-23,2135,1123/2020,2020-12-22,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,156,18.0,22.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2990,277.0,430.0,PROCESSO ADMINISTRATIVO SANCIONADOR,771969
8,1,usi,2020-12-24,2136,36565,2020-12-24,BACEN,Divulga nome de pessoas com intenção de ocupar...,140,19.0,21.0,Divulgamos os nomes de pretendentes a cargos d...,1574,149.0,280.0,COMUNICADO,772675
9,1,usi,2020-12-24,2137,1061/2020,2020-12-23,BACEN,Caracterizado fornecimento intempestivo de in...,467,57.0,88.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,859,97.0,146.0,PROCESSO ADMINISTRATIVO SANCIONADOR,772640


In [5]:
bbrc.tail(20)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id
61844,0,uni,2020-07-28,112246,1654,2020-06-19,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Inclui o art. 2º-B e altera os arts. 3º e 5º d...,284,36.0,41.0,"A CONTROLADORA-GERAL DO MUNICÍPIO, no uso das ...",4359,363.0,706.0,RESOLUÇÃO,707723
61845,0,uni,2020-07-28,112247,3163,2020-06-18,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre operações de fiscalização das\r\n...,69,9.0,9.0,"A SECRETÁRIA MUNICIPAL DE FAZENDA, no uso das ...",2668,266.0,456.0,RESOLUÇÃO,707691
61846,0,uni,2020-07-28,112248,47533,2020-06-11,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,"Altera o Decreto Rio nº 47.282, de 21 de março...",213,29.0,31.0,"O PREFEITO DA CIDADE DO RIO DE JANEIRO, no uso...",4825,270.0,476.0,DECRETO,705560
61847,0,uni,2020-07-28,112249,57,2020-06-10,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,"/Dispõe sobre a prorrogação da licença médica,...",59,9.0,9.0,O SUBSECRETÁRIO DE SERVIÇOS COMPARTILHADOS DA ...,1370,155.0,220.0,PORTARIA,705564
61848,0,uni,2020-07-28,112250,1002,2020-06-10,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Prorroga o prazo de validade das certidões de\...,86,14.0,14.0,O PROCURADOR-GERAL DO MUNICÍPIO DO RIO DE JANE...,2268,188.0,370.0,RESOLUÇÃO,706277
61849,0,uni,2020-07-28,112251,157,2020-06-05,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Altera a composição da Comissão de\r\nPrograma...,104,15.0,16.0,A PRESIDENTE DA COMISSÃO DE PROGRAMAÇÃO E CONT...,642,86.0,111.0,DELIBERAÇÃO,703886
61850,0,dicre,2021-11-09,57669,7.107,2021-11-04,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre a margem consignável dos servidor...,243,28.0,31.0,Legislação - Lei Ordinária Lei nº 7107/2021 Da...,1963,207.0,307.0,LEI,877367
61851,0,dicre,2021-10-28,57670,49695,2021-10-27,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Cria a Área de Proteção Ambiental do Sertão Ca...,52,9.0,9.0,"DECRETO RIO Nº 49695, DE 27 DE OUTUBRO DE 2021...",18456,879.0,2932.0,DECRETO,873815
61852,0,dicre,2021-10-28,57671,49696,2021-10-27,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Estabelece o zoneamento da área de proteção am...,159,24.0,27.0,"DECRETO RIO Nº 49696, DE 27 DE OUTUBRO DE 2021...",16436,872.0,2721.0,DECRETO,873814
61853,0,uac,2021-04-26,50249,48768,2021-04-23,CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO,Dispõe sobre as competências da Empresa Munici...,81,12.0,12.0,DECRETO RIO Nº 48768 DE 23 DE ABRIL DE 2021 Di...,65853,1963.0,9767.0,DECRETO,809105


<h3>Evaluate which regulators get in each department to make the intra-class balancing strategy</h3>

In [6]:
lt = ['usi', 'cib', 'bb_asset', 'dimep', 'coger gesub', 'disem']

In [7]:
data_relevant = bbrc[bbrc['class'] == 1]

In [8]:
for l in lt:
    dfr = data_relevant[data_relevant.department.isin([l])]
    dddd = dict(dfr.regulatory_authority.value_counts())
    print(l,' - ', list(dict(sorted(dddd.items(), key=lambda item: item[1], reverse=True)).items())[:6])

usi  -  [('BACEN', 666), ('PRESIDÊNCIA_DA_REPÚBLICA', 29), ('COAF', 28), ('DOU', 26), ('ANAC', 14), ('RFB', 14)]
cib  -  [('B3', 302), ('BACEN', 175), ('CETIP', 99), ('ANBIMA', 55), ('CVM', 49), ('RFB', 7)]
bb_asset  -  [('CVM', 253), ('ANBIMA', 221), ('B3', 147), ('COAF', 18), ('BACEN', 16), ('RFB', 13)]
dimep  -  [('BACEN', 205), ('CIP', 198), ('DOU', 64), ('BNDES', 55), ('PRESIDÊNCIA_DA_REPÚBLICA', 32), ('NUCLEA', 17)]
coger gesub  -  [('BACEN', 193), ('CVM', 126), ('DOU', 107), ('RFB', 79), ('CFC', 22), ('CPC', 13)]
disem  -  [('B3', 97), ('CETIP', 85), ('BACEN', 68), ('STN', 67), ('RFB', 24), ('ANBIMA', 20)]


In [9]:
# We chose the regulators with more documents in the relevant class of each of the three departments.
# The regulators must exist in both classes of the department.
# Only regulators with more than 10 documents in the relevant class where chosen.
# To make the undersampling, we took the regulator with less documents in the relevant class for each department, 26 documents at DPT_25, for example.

# usi  -  [('BACEN', 666), ('PRESIDÊNCIA_DA_REPÚBLICA', 29), ('COAF', 28), ('DOU', 26)] # relevant
# usi  -  [('DOU', 2982), ('BACEN', 467), ('COAF', 48), ('PRESIDÊNCIA_DA_REPÚBLICA', 32)] # irrelevant

# cib  -  [('B3', 302), ('BACEN', 175), ('ANBIMA', 55), ('CVM', 49)] # relevant
# cib  -  [('B3', 592), ('CVM', 368), ('BACEN', 257), ('ANBIMA', 85)] # irrelevant

# bb_asset  -  [('CVM', 253), ('ANBIMA', 221), ('B3', 147), ('RFB', 13)] # relevant
# bb_asset  -  [('B3', 1921), ('CVM', 783), ('RFB', 678), ('ANBIMA', 399)] # irrelevant

In [10]:
data_irrelevant = bbrc[bbrc['class'] == 0]

In [11]:
for l in lt:
    dfi = data_irrelevant[data_irrelevant.department.isin([l])]
    dddd = dict(dfi.regulatory_authority.value_counts())
    print(l,' - ',list(dict(sorted(dddd.items(), key=lambda item: item[1], reverse=True)).items())[:6])

usi  -  [('DOU', 2982), ('BACEN', 467), ('ANAC', 421), ('COAF', 48), ('PRESIDÊNCIA_DA_REPÚBLICA', 32), ('NUCLEA', 26)]
cib  -  [('B3', 592), ('CVM', 368), ('STN', 264), ('BACEN', 257), ('RFB', 161), ('ANBIMA', 85)]
bb_asset  -  [('B3', 1921), ('CVM', 783), ('CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO', 744), ('RFB', 678), ('ANBIMA', 399), ('SUSEP', 391)]
dimep  -  [('BACEN', 826), ('CIP', 712), ('CVM', 531), ('RFB', 331), ('STN', 269), ('DOU', 230)]
coger gesub  -  [('DOU', 1147), ('BACEN', 658), ('RFB', 327), ('PRESIDÊNCIA_DA_REPÚBLICA', 201), ('CVM', 158), ('NUCLEA', 60)]
disem  -  [('BACEN', 337), ('STN', 294), ('RFB', 270), ('CVM', 162), ('B3', 121), ('CAMARA_MUNICIPAL_DO_RIO_DE_JANEIRO', 101)]


In [12]:
data_relevant = bbrc[bbrc['class'] == 1]

In [13]:
aux_relevant = []

SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25 = 26
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3 = 49
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2 = 13

# usi
usi = data_relevant[data_relevant.department.isin(['usi'])]

regulator = usi[usi.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = usi[usi.regulatory_authority.isin(['PRESIDÊNCIA_DA_REPÚBLICA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = usi[usi.regulatory_authority.isin(['COAF'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)

regulator = usi[usi.regulatory_authority.isin(['DOU'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_relevant.append(regulator)
# usi

# cib
cib = data_relevant[data_relevant.department.isin(['cib'])]

regulator = cib[cib.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = cib[cib.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = cib[cib.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)

regulator = cib[cib.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_relevant.append(regulator)
# cib

# bb_asset
bb_asset = data_relevant[data_relevant.department.isin(['bb_asset'])]

regulator = bb_asset[bb_asset.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = bb_asset[bb_asset.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = bb_asset[bb_asset.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)

regulator = bb_asset[bb_asset.regulatory_authority.isin(['RFB'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_relevant.append(regulator)
# bb_asset

data_relevant = pd.concat(aux_relevant)

data_relevant[data_relevant.text_words < 50]

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id


In [14]:
data_relevant.shape

(352, 17)

In [15]:
(13*4)+(49*4)+(26*4)

352

In [16]:
aux_irrelevant = []

SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25 = 26
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3 = 49
SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2 = 13

# usi
usi = data_irrelevant[data_irrelevant.department.isin(['usi'])]

regulator = usi[usi.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = usi[usi.regulatory_authority.isin(['PRESIDÊNCIA_DA_REPÚBLICA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = usi[usi.regulatory_authority.isin(['COAF'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)

regulator = usi[usi.regulatory_authority.isin(['DOU'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_25]
aux_irrelevant.append(regulator)
# usi

# cib
cib = data_irrelevant[data_irrelevant.department.isin(['cib'])]

regulator = cib[cib.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = cib[cib.regulatory_authority.isin(['BACEN'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = cib[cib.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)

regulator = cib[cib.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_3]
aux_irrelevant.append(regulator)
# cib

# bb_asset
bb_asset = data_irrelevant[data_irrelevant.department.isin(['bb_asset'])]

regulator = bb_asset[bb_asset.regulatory_authority.isin(['CVM'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = bb_asset[bb_asset.regulatory_authority.isin(['ANBIMA'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = bb_asset[bb_asset.regulatory_authority.isin(['B3'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)

regulator = bb_asset[bb_asset.regulatory_authority.isin(['RFB'])]
regulator = regulator[regulator.text_words >= 50][:SMALLEST_QUANTITY_RELEVANT_SAMPLES_DPT_2]
aux_irrelevant.append(regulator)
# bb_asset

data_irrelevant = pd.concat(aux_irrelevant)

data_irrelevant[data_irrelevant.text_words < 50]

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id


In [17]:
data_irrelevant.shape

(352, 17)

In [18]:
data_irrelevant.head(20)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id
7846,0,usi,2020-08-31,2169,0301/2020,2020-08-31,BACEN,Caracterizado fornecimento intempestivo ao Ba...,294,36.0,49.0,​DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONAD...,758,81.0,113.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729080
7847,0,usi,2020-08-31,2170,36121,2020-08-31,BACEN,Divulga nome aprovado de pessoas eleitas/nomea...,167,21.0,23.0,Divulgamos o nome aprovado de pessoas eleitas/...,7080,407.0,966.0,COMUNICADO,729131
7848,0,usi,2020-08-31,2171,36123,2020-08-31,BACEN,Divulga nome de pessoas com intenção de ocupar...,140,19.0,21.0,Divulgamos os nomes de pretendentes a cargos d...,1558,149.0,276.0,COMUNICADO,729151
7849,0,usi,2020-08-31,2172,705/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2930,275.0,432.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729089
7850,0,usi,2020-08-31,2173,703/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2930,277.0,432.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729091
7851,0,usi,2020-08-31,2174,701/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2920,275.0,429.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729093
7852,0,usi,2020-08-31,2175,700/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2871,271.0,424.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729094
7853,0,usi,2020-08-31,2176,704/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2875,271.0,426.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729090
7854,0,usi,2020-08-31,2177,702/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2913,276.0,430.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729092
7855,0,usi,2020-08-31,2178,699/2020,2020-08-30,BACEN,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,162,19.0,23.0,DEPARTAMENTO DE RESOLUÇÃO E DE AÇÃO SANCIONADO...,2866,271.0,425.0,PROCESSO ADMINISTRATIVO SANCIONADOR,729095


In [19]:
data_irrelevant.tail(20)

,class,department,entry_date,general_id,normative_identifier,publication_date,regulatory_authority,subject,subject_length,subject_unique_words,subject_words,text,text_length,text_unique_words,text_words,type,unique_document_id
20082,0,bb_asset,2021-01-12,175,005-2021-VOP-Comunicado Externo,2021-01-12,B3,Adriano Domingues Sanches (CPF/MF 213.928.908-...,1380,115.0,161.0,12 de janeiro de 2021 005/2021-VOP COMUNICADO ...,2143,191.0,273.0,COMUNICADO,777556
20083,0,bb_asset,2021-01-12,176,004-2021-VOP-Comunicado Externo,2021-01-12,B3,"A B3 informa que, no processamento batch de 13...",317,41.0,50.0,12 de janeiro de 2021 004/2021-VOP COMUNICADO ...,1319,148.0,195.0,COMUNICADO,777557
20084,0,bb_asset,2021-01-12,177,003-2021-VPC-Ofício Circular,2021-01-12,B3,"Para o presente programa, serão credenciados a...",150,22.0,22.0,12 de janeiro de 2021 003/2021-VPC OFÍCIO CIRC...,15647,718.0,2383.0,OFÍCIO CIRCULAR,777568
20085,0,bb_asset,2021-01-14,195,007-2021-VOP-Comunicado Externo,2021-01-14,B3,Felipe Silva de Oliveira (CPF/MF 124.623.267-7...,624,57.0,76.0,14 de janeiro de 2021 007/2021-VOP COMUNICADO ...,1387,134.0,188.0,COMUNICADO,778402
20086,0,bb_asset,2021-01-14,196,006-2021-VOP-Comunicado Externo,2021-01-14,B3,Visando auxiliar o planejamento de mudanças e ...,298,35.0,43.0,14 de janeiro de 2021 006/2021-VOP COMUNICADO ...,1643,162.0,235.0,COMUNICADO,778403
20087,0,bb_asset,2021-01-19,261,008-2021-VOP-Comunicado Externo,2021-01-19,B3,Adercio Jose Ferreira (CPF/MF 001.153.771-07) ...,151,18.0,20.0,19 de janeiro de 2021 008/2021-VOP COMUNICADO ...,796,94.0,116.0,COMUNICADO,779913
20088,0,bb_asset,2021-01-19,262,002-2021-PRE-Ofício Circular,2021-01-19,B3,"Informamos que, em 26/01/2021, entrarão em vig...",136,19.0,21.0,19 de janeiro de 2021 002/2021-PRE OFÍCIO CIRC...,4145,257.0,619.0,OFÍCIO CIRCULAR,779914
42624,0,bb_asset,2020-12-30,13,Publicação do Ato Declaratório Executivo Cofis...,2020-12-30,RFB,Ato Declaratório Executivo Cofis nº 86/2020,43,6.0,6.0,Publicado em 30/12/2020 Ato Declaratório Execu...,823,94.0,127.0,COMUNICADO,774177
42625,0,bb_asset,2020-12-30,14,Nova versão dos leiautes da EFD-Reinf,2020-12-30,RFB,Foi publicada a versão 1.5.1 dos Leiautes da E...,114,16.0,18.0,Publicado em 30/12/2020 Foi publicada a versão...,343,47.0,52.0,COMUNICADO,774178
42626,0,bb_asset,2020-12-30,15,Publicação da versão 1.1.5 do Manual da e-fina...,2020-12-30,RFB,"ADE COFIS Nº 85, de 28 de dezembro de 2020, ap...",90,15.0,17.0,"Publicado em 30/12/2020 ADE COFIS Nº 85, de 28...",7209,454.0,1151.0,COMUNICADO,774179


In [20]:
data_relevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
dtype: int64

In [21]:
data_relevant = data_relevant[data_relevant.text_words >= 50]
data_relevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
dtype: int64

In [22]:
data_irrelevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
dtype: int64

In [23]:
data_irrelevant = data_irrelevant[data_irrelevant.text_words >= 50]
data_irrelevant.count()

class                   352
department              352
entry_date              352
general_id              352
normative_identifier    352
publication_date        352
regulatory_authority    352
subject                 352
subject_length          352
subject_unique_words    352
subject_words           352
text                    352
text_length             352
text_unique_words       352
text_words              352
type                    352
unique_document_id      352
dtype: int64

In [24]:
data_relevant['department'].unique()

array(['usi', 'cib', 'bb_asset'], dtype=object)

In [25]:
len(list(data_relevant['department'].unique()))

3

In [26]:
stop_words = set(stopwords.words('portuguese'))
df_stopwords = pd.DataFrame(list(stop_words), columns =['words'])
df_stopwords.to_csv('stop_words_nltk_portuguese.csv', sep=';', encoding="utf-8")
df_stopwords.head(3)

,words
0,teve
1,formos
2,estivessem


In [27]:
def remove_special_characters(df,column):
    df.loc[:,column] = df[column].str.lower()
    
    df.loc[:,column] = df[column].str.replace('á','a')
    df.loc[:,column] = df[column].str.replace('é','e')
    df.loc[:,column] = df[column].str.replace('í','i')
    df.loc[:,column] = df[column].str.replace('ó','o')
    df.loc[:,column] = df[column].str.replace('ú','u')

    df.loc[:,column] = df[column].str.replace('â','a')
    df.loc[:,column] = df[column].str.replace('ê','e')
    df.loc[:,column] = df[column].str.replace('î','i')
    df.loc[:,column] = df[column].str.replace('ô','o')
    df.loc[:,column] = df[column].str.replace('û','u')

    df.loc[:,column] = df[column].str.replace('à','a')
    df.loc[:,column] = df[column].str.replace('è','e')
    df.loc[:,column] = df[column].str.replace('ì','i')
    df.loc[:,column] = df[column].str.replace('ò','o')
    df.loc[:,column] = df[column].str.replace('ù','u')

    df.loc[:,column] = df[column].str.replace('ä','a')
    df.loc[:,column] = df[column].str.replace('ë','e')
    df.loc[:,column] = df[column].str.replace('ï','i')
    df.loc[:,column] = df[column].str.replace('ö','o')
    df.loc[:,column] = df[column].str.replace('ü','u')

    df.loc[:,column] = df[column].str.replace('ã','a')
    df.loc[:,column] = df[column].str.replace('ẽ','e')
    df.loc[:,column] = df[column].str.replace('ĩ','i')
    df.loc[:,column] = df[column].str.replace('õ','o')
    df.loc[:,column] = df[column].str.replace('ũ','u')

    df.loc[:,column] = df[column].str.replace('ç','c')
    
    return df

In [28]:
df_stopwords = remove_special_characters(df_stopwords,'words')

In [29]:
df_stopwords.head(20)

,words
0,teve
1,formos
2,estivessem
3,tem
4,tivessem
5,terao
6,ela
7,entre
8,houverao
9,num


In [30]:
df_stopwords.tail(20)

,words
187,tenham
188,terei
189,sou
190,esteve
191,fosse
192,de
193,houvessem
194,houvera
195,aquelas
196,este


In [31]:
stop_words = df_stopwords['words'].tolist()

In [32]:
def clean_text(text, stop_words):
    text = [token for token in text.split(" ")]
    text = [word for word in text if not word in stop_words] # remove stopwords
    text = " ".join(text)
    return text

In [33]:
def cleaning(df,column, stop_words):
    # Remove urls
    df.loc[:,column] = df[column].replace(r'https?:\/\/.*?[\s+]', ' ', regex=True)
    # Remove urls without https?
    df.loc[:,column] = df[column].replace(r'www.*?[\s+]', ' ', regex=True)
    # Remove email address
    df.loc[:,column] = df[column].replace(r'(?P<email_address>[\w\.-]+@[\w\.-]+\.[\w]+)', ' ', regex=True)
    
    df.loc[:,column] = df[column].str.replace('º',' ')
    df.loc[:,column] = df[column].str.replace('ª',' ')

    df.loc[:,column] = df[column].str.replace('1',' ')
    df.loc[:,column] = df[column].str.replace('2',' ')
    df.loc[:,column] = df[column].str.replace('3',' ')
    df.loc[:,column] = df[column].str.replace('4',' ')
    df.loc[:,column] = df[column].str.replace('5',' ')
    df.loc[:,column] = df[column].str.replace('6',' ')
    df.loc[:,column] = df[column].str.replace('7',' ')
    df.loc[:,column] = df[column].str.replace('8',' ')
    df.loc[:,column] = df[column].str.replace('9',' ')
    df.loc[:,column] = df[column].str.replace('0',' ')

    df.loc[:,column] = df[column].str.replace('/',' ')

    df.loc[:,column] = df[column].str.replace('\r',' ')
    df.loc[:,column] = df[column].str.replace('\n',' ')
    df.loc[:,column] = df[column].str.replace('\t',' ')
    df.loc[:,column] = df[column].str.replace('\\',' ')

    df.loc[:,column] = df[column].str.replace('-',' ')

    df.loc[:,column] = df[column].str.replace('–',' ')
    df.loc[:,column] = df[column].str.replace('“',' ')
    df.loc[:,column] = df[column].str.replace('”',' ')
    df.loc[:,column] = df[column].str.replace('’',' ')
    df.loc[:,column] = df[column].str.replace('_',' ')
    df.loc[:,column] = df[column].str.replace('.',' ')
    df.loc[:,column] = df[column].str.replace(',',' ')
    df.loc[:,column] = df[column].str.replace('|',' ')
    df.loc[:,column] = df[column].str.replace('=',' ')
    df.loc[:,column] = df[column].str.replace('@',' ')
    df.loc[:,column] = df[column].str.replace('$',' ')

    df.loc[:,column] = df[column].str.replace('°',' ')
    df.loc[:,column] = df[column].str.replace('§',' ')
    df.loc[:,column] = df[column].str.replace('•',' ')
    df.loc[:,column] = df[column].str.replace('▪',' ')

    df.loc[:,column] = df[column].str.replace('%',' ')
    df.loc[:,column] = df[column].str.replace('&',' ')
    df.loc[:,column] = df[column].str.replace('*',' ')
    df.loc[:,column] = df[column].str.replace('+',' ')
    df.loc[:,column] = df[column].str.replace(':',' ')
    df.loc[:,column] = df[column].str.replace(';',' ')

    df.loc[:,column] = df[column].str.replace('!',' ')
    df.loc[:,column] = df[column].str.replace('?',' ')
    df.loc[:,column] = df[column].str.replace('#',' ')
    df.loc[:,column] = df[column].str.replace('\'',' ')
    df.loc[:,column] = df[column].str.replace('"',' ')

    df.loc[:,column] = df[column].str.replace('<',' ')
    df.loc[:,column] = df[column].str.replace('>',' ')

    df.loc[:,column] = df[column].str.replace('(',' ')
    df.loc[:,column] = df[column].str.replace(')',' ')

    df.loc[:,column] = df[column].str.replace('{',' ')
    df.loc[:,column] = df[column].str.replace('}',' ')

    df.loc[:,column] = df[column].str.replace('[',' ')
    df.loc[:,column] = df[column].str.replace(']',' ')

    # Remove the same character repeated more than twice
    df.loc[:,column] = df[column].replace(r'([a-z])\1{2,}', ' ', regex=True)
    
    # To replace more than one white space by only one white space
    df.loc[:,column] = df[column].replace(r'\s+', ' ', regex=True)

    # Remove stopwords
    df.loc[:,column] = df[column].apply(lambda x: clean_text(x, stop_words))

    return df

In [34]:
def remove_duplicates(sentence):
	words = sentence.split(" ")
	result = []
	for word in words:
		if word not in result:
			result.append(word)
	return result

<h3>Selecting the Departments</h3>

In [35]:
boards = data_relevant['department']
# Getting the 6 most populated boards
boards_list = list(boards.unique())[:DEPARTMENTS_QUANTITY_TO_TRAIN]
lg.info(f"boards_list: {boards_list}")

2024-03-25 22:04:31.342 | INFO     | __main__:<module>:4 - boards_list: ['usi', 'cib', 'bb_asset']


<h3>Parameters</h3>

In [36]:
from sklearn.metrics import make_scorer
import sklearn.metrics as sk

CROSS_VALIDATION=10
METRICS = {
           'accuracy': make_scorer(sk.accuracy_score),
           'precision': make_scorer(sk.precision_score, average = 'weighted', zero_division=0),
           'recall': make_scorer(sk.recall_score, average = 'weighted', zero_division=0),
           'f1': make_scorer(sk.f1_score, average = 'binary', zero_division=0),
           'f1_macro': make_scorer(sk.f1_score, average = 'macro', zero_division=0),
           'f1_weighted': make_scorer(sk.f1_score, average = 'weighted', zero_division=0)
           }

RAND_STATE = 42
REFIT_VAL = 'f1'

cls = dict({
        'random_forest': {
            'estimator': RandomForestClassifier(random_state=RAND_STATE, n_jobs = NUM_PROCESSORS),
            'parameters':{'n_estimators': [100, 400, 1000], 'max_depth': [10,30, 100], 'criterion':['entropy', 'log_loss', 'gini']}
            },
        'svm': {
            'estimator': SVC(random_state=RAND_STATE),
            'parameters':{'C': [0.025, 0.08, 0.1, 0.5, 0.8, 1.0, 2.0, 10.0, 100.0, 500.0, 1000.0], 'kernel': ['linear','poly','rbf','sigmoid']}
            },
        'xgboost':{
            'estimator': XGBClassifier(random_state=RAND_STATE, num_class=2, verbosity = 0, silent=True, n_jobs=NUM_PROCESSORS),
            'parameters':{'objective':['reg:squarederror','binary:logistic','multi:softmax','binary:hinge'],'n_estimators': [100,1000],'max_depth': [10,30],'learning_rate':[0.01,0.5]}
        },
        'nb':{
                'estimator': MultinomialNB(),
                'parameters':{'alpha':[1, 0.1, 0.01, 0.001, 0.0001, 0.00001],'force_alpha': [True, False],'fit_prior': [True, False]}
            }
    })

In [37]:
from sklearn.model_selection import train_test_split
from sklearn import metrics

def training_each_inductor_holdout(classifiers, board, X, y):
    for key in list(classifiers.keys()):
        lg.info(f"\n\n####### {key} INDUCTOR EXECUTING ####### - {board}\n\n")

        lg.debug(f"[+] classifiers[key]['estimator']: {classifiers[key]['estimator']} - {key} - {board}")
        lg.debug(f"[+] classifiers[key]['parameters']: {classifiers[key]['parameters']} - {key} - {board}")

        search = GridSearchCV(estimator=classifiers[key]['estimator'],
                            param_grid=classifiers[key]['parameters'],
                            cv = CROSS_VALIDATION,
                            scoring = METRICS,
                            refit = REFIT_VAL,
                            n_jobs = NUM_PROCESSORS,
                            verbose = VERBOSE)

        lg.debug(f"[+] Start GridSearchCV - {key} - {board}")
        search.fit(X, y)
        lg.debug(f"[+] End GridSearchCV - {key} - {board}")

        lg.info(f"search.best_estimator_: {search.best_estimator_} \n")

        x_train, x_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=RAND_STATE, stratify=y)

        clf = search.best_estimator_.fit(x_train, y_train)
        
        y_pred = clf.predict(x_test)

        lg.success(f"\n\n####### METRICS {key} - {board} #######\n\n")

        lg.success(f"{key} - {board} - test_precision: {metrics.precision_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_recall: {metrics.recall_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_f1_binary: {metrics.f1_score(y_test, y_pred)}")

        lg.success(f"{key} - {board} - test_f1_macro: {metrics.f1_score(y_test, y_pred, average='macro')}")

<h3>The Training and Evaluation</h3>

In [ ]:
for board in boards_list:
    lg.info(f"\n\n####### {board} EXECUTING #######\n\n")
    total_relevant_documents_of_the_board = data_relevant[data_relevant.department == board].shape[0]
    lg.info(f"total_relevant_documents_of_the_board: {total_relevant_documents_of_the_board} - {board}")
    lg.info(f"Total irrelevant documents of the department: {data_irrelevant[data_irrelevant.department == board].shape[0]}")

    data_relevant_board = data_relevant[data_relevant.department == board]
    lg.info(f"data_relevant_board: {data_relevant_board.shape} - {board}")

    # The total documents of the relevant class (total_relevant_documents_of_the_board) is use to get the first documents in the irrelevant class.
    data_irrelevant_board = data_irrelevant[data_irrelevant.department == board][:total_relevant_documents_of_the_board]
    lg.info(f"data_irrelevant_board: {data_irrelevant_board.shape} - {board}")

    df_gather_datasets = []
    df_gather_datasets.append(data_relevant_board)
    df_gather_datasets.append(data_irrelevant_board)
    # Merging all dataframes in the list as one dataframe.
    df_final_dataset = pd.concat(df_gather_datasets)
    
    lg.info(f"df_final_dataset: {df_final_dataset.shape} - {board}")

    # Cleaning the text and title
    lg.debug("[+] Start cleaning text and title")
    df_final_dataset = remove_special_characters(df_final_dataset, 'text')
    df_final_dataset = remove_special_characters(df_final_dataset, 'subject')
    df_final_dataset = cleaning(df_final_dataset, 'text', stop_words)
    df_final_dataset = cleaning(df_final_dataset, 'subject', stop_words)
    lg.debug("[+] End cleaning text and title")

    # To get only phrases with at lest three words after the preprocessing.
    df_final_dataset = df_final_dataset[df_final_dataset['text'].str.len() >= 3]
    
    data_x = df_final_dataset.text
    lg.debug("[+] Start TF-IDF")

    feature_extraction = TfidfVectorizer()
    data_x = feature_extraction.fit_transform(data_x)
    lg.debug("[+] End TF-IDF")

    data_y = df_final_dataset['class']

    training_each_inductor_holdout(cls, board, data_x, data_y)

    lg.success("\n\n[+][+][+] SUCCESS! [+][+][+]\n\n")